# Projet 3 : Concevez une application au service de la santé publique

## Partie 1 : nettoyage du jeux de données

In [1]:
import pandas as pd



### Ouverture du fichier .csv

In [2]:
# ouverture du fichire csv
OpenFoodData = pd.read_csv('./en.openfoodfacts.org.products.csv',
                           sep='\t', # séparateurs = tabulations
                           low_memory=False)


In [3]:
# 3 premières lignes du fichier
OpenFoodData.head(3)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,abbreviated_product_name,generic_name,...,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,00000000000000225,http://world-en.openfoodfacts.org/product/0000...,nutrinet-sante,1623855208,2021-06-16T14:53:28Z,1623855209,2021-06-16T14:53:29Z,jeunes pousses,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00000000000003429145,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1630483911,2021-09-01T08:11:51Z,1630484064,2021-09-01T08:14:24Z,L.casei,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,Vitória crackers,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Entêtes des colonnes

In [4]:
# liste des entêtes de colonnes
OpenFoodData.columns.to_list()

['code',
 'url',
 'creator',
 'created_t',
 'created_datetime',
 'last_modified_t',
 'last_modified_datetime',
 'product_name',
 'abbreviated_product_name',
 'generic_name',
 'quantity',
 'packaging',
 'packaging_tags',
 'packaging_text',
 'brands',
 'brands_tags',
 'categories',
 'categories_tags',
 'categories_en',
 'origins',
 'origins_tags',
 'origins_en',
 'manufacturing_places',
 'manufacturing_places_tags',
 'labels',
 'labels_tags',
 'labels_en',
 'emb_codes',
 'emb_codes_tags',
 'first_packaging_code_geo',
 'cities',
 'cities_tags',
 'purchase_places',
 'stores',
 'countries',
 'countries_tags',
 'countries_en',
 'ingredients_text',
 'allergens',
 'allergens_en',
 'traces',
 'traces_tags',
 'traces_en',
 'serving_size',
 'serving_quantity',
 'no_nutriments',
 'additives_n',
 'additives',
 'additives_tags',
 'additives_en',
 'ingredients_from_palm_oil_n',
 'ingredients_from_palm_oil',
 'ingredients_from_palm_oil_tags',
 'ingredients_that_may_be_from_palm_oil_n',
 'ingredients_t

Il y a plusieurs types de colonnes contenant des dates : 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime'. Les colonnes finissant par "_t" sont en timestamp c'est à dire le nombre de secondes écoulées debuis le 1er janvier 1970 (début de l'heure UNIX). Les colonnes finissant par "_datetime" sont au format : yyyy-mm-ddThh:mn:ssZ.

Nous allons supprimer les timestamps et mettre au format date les colonnes datetime

In [82]:
OpenFoodData.loc[:, OpenFoodData.columns.str.endswith('_t')].head(3)

,created_t,last_modified_t
0,1623855208,1623855209
1,1630483911,1630484064
2,1529059080,1561463718


In [83]:
OpenFoodData.loc[:, OpenFoodData.columns.str.endswith('_datetime')].head(3)

,created_datetime,last_modified_datetime
0,2021-06-16T14:53:28Z,2021-06-16T14:53:29Z
1,2021-09-01T08:11:51Z,2021-09-01T08:14:24Z
2,2018-06-15T10:38:00Z,2019-06-25T11:55:18Z


In [87]:
# suppression timestamp
OFDClean = OpenFoodData.loc[:, ~OpenFoodData.columns.str.endswith('_t')]
# mise au format temps
OFDClean.loc[:, OFDClean.columns.str.endswith('_datetime')].apply(
    lambda x: pd.to_datetime(x, infer_datetime_format=True))


,created_datetime,last_modified_datetime
0,2021-06-16 14:53:28,2021-06-16 14:53:29
1,2021-09-01 08:11:51,2021-09-01 08:14:24
2,2018-06-15 10:38:00,2019-06-25 11:55:18
3,2018-10-13 21:06:14,2018-10-13 21:06:57
4,2019-11-19 15:02:16,2021-06-22 19:39:25
...,...,...
1985069,2019-10-31 09:24:26,2019-10-31 09:24:26
1985070,2020-12-16 07:58:23,2020-12-16 07:58:24
1985071,2020-02-08 14:20:13,2020-02-25 15:24:07
1985072,2021-02-12 11:35:28,2021-02-12 11:35:30
